In [65]:
%matplotlib inline

import os
import numpy as np
import tensorflow as tf
import tensorflow.keras 
import matplotlib.pyplot as plt
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.datasets import cifar100


In [66]:
batch_size = 128
buffersize = 50000
steps_per_epoch = int(np.ceil(50000) / float(batch_size))
epochs = 1000
num_classes = 100
validation_split = 0.2

(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = cifar100.load_data(label_mode="fine")

X_train = X_train_orig.astype(np.float32)/255
X_test = X_test_orig.astype(np.float32)/255
Y_train = tf.one_hot(Y_train_orig, num_classes)
Y_train = tf.reshape(Y_train, (Y_train.shape[0], Y_train.shape[2]))
Y_test = tf.one_hot(Y_test_orig, num_classes)
Y_test = tf.reshape(Y_test, (Y_test.shape[0], Y_test.shape[2]))
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(Y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(Y_test.shape))

X_train: (50000, 32, 32, 3)
Y_train: (50000, 100)
X_test:  (10000, 32, 32, 3)
Y_test:  (10000, 100)


In [67]:
def Conv_Model(input_shape):
    Conv_Model = models.Sequential()
    Conv_Model.add(layers.Conv2D(32, (3,3), input_shape = input_shape))
    Conv_Model.add(layers.LeakyReLU())
    Conv_Model.add(layers.Dropout(0.3))

    Conv_Model.add(layers.MaxPooling2D((2,2)))

    Conv_Model.add(layers.Conv2D(64, (3,3)))
    Conv_Model.add(layers.LeakyReLU())
    Conv_Model.add(layers.Dropout(0.3))

    Conv_Model.add(layers.MaxPooling2D((2,2)))

    Conv_Model.add(layers.Conv2D(128, (3,3)))
    Conv_Model.add(layers.LeakyReLU())
    Conv_Model.add(layers.Dropout(0.3))

    Conv_Model.add(layers.MaxPooling2D((2,2)))

    Conv_Model.add(layers.Flatten())
    Conv_Model.add(layers.Dense(256, activation='relu'))
    Conv_Model.add(layers.Dense(128, activation='relu'))
    Conv_Model.add(layers.Dense(100, activation='softmax'))

    return Conv_Model

In [68]:
Conv_Model = Conv_Model(X_train.shape[1:])
Conv_Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
Conv_Model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_33 (Dropout)         (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 13, 13, 64)      

In [69]:
history = Conv_Model.fit(X_train, Y_train, batch_size= batch_size, epochs= epochs, verbose = 1, validation_split=validation_split)
model.save("imgnet_v2_cnn_epo:" + str(epochs) + "_btchsz:" + str(batch_size) + ".h5")

Epoch 1/1000
313/313 [==============================] - 23s 75ms/step - loss: 4.0937 - accuracy: 0.0715 - val_loss: 3.8776 - val_accuracy: 0.1089
Epoch 2/1000
313/313 [==============================] - 24s 76ms/step - loss: 3.4569 - accuracy: 0.1716 - val_loss: 3.5459 - val_accuracy: 0.1659
Epoch 3/1000
313/313 [==============================] - 27s 87ms/step - loss: 3.1001 - accuracy: 0.2385 - val_loss: 3.2407 - val_accuracy: 0.2235
Epoch 4/1000
313/313 [==============================] - 29s 93ms/step - loss: 2.8517 - accuracy: 0.2887 - val_loss: 3.1707 - val_accuracy: 0.2298
Epoch 5/1000
313/313 [==============================] - 27s 85ms/step - loss: 2.6773 - accuracy: 0.3211 - val_loss: 3.1553 - val_accuracy: 0.2390
Epoch 6/1000
313/313 [==============================] - 26s 83ms/step - loss: 2.5224 - accuracy: 0.3540 - val_loss: 2.9681 - val_accuracy: 0.2708
Epoch 7/1000
313/313 [==============================] - 26s 83ms/step - loss: 2.4000 - accuracy: 0.3803 - val_loss: 3.0342 -

KeyboardInterrupt: 

In [ ]:
score = Conv_Model.evaluate(X_test, Y_test, verbose=1)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

plt.plot(history.history['val_loss'])
plt.title('Validation loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

plt.plot(history.history['val_accuracy'])
plt.title('Validation accuracy history')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()